In [4]:
cd ../

c:\Progetto_Ing_Informatica\OLA_2023_Private


In [28]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import deque

from User_Classes import UserClass
from Advertising_Environment import calculate_margin, calculate_price_index
from Clairvoyant import find_optimal_bid_for_class
from Advertising.GPTS_Learner import GPTS_Learner
from Advertising.GPUCB_Learner import GPUCB_Learner
from TS_Learner import TS_Learner

import warnings
warnings.filterwarnings("ignore")

# Environment

In [21]:
#Environment
n_arms = 100

prices = [50, 100, 150, 200, 250]
bids = np.linspace(0.01, 3.0, n_arms)
sigma = 200

user = UserClass(name='C1')

T = 20
n_experiments = 5

gpts_rewards_per_experiment = []
gpucb_rewards_per_experiment = []


optimum_bid = find_optimal_bid_for_class(user.user_index, calculate_price_index(prices[3]))

In [7]:
optimum_bid

(1.0, 13767.97338369385)

In [8]:
class Environment_Pricing_3(): #m
    def __init__(self, n_arms, user_class, probabilities):
        self.user_class = user_class
        self.time = 0
        self.n_arms = n_arms
        self.probabilities = probabilities

    def round(self, user_class, price):
        successes = np.random.binomial(n_trials, user_class.get_conversion_probabilities(price))
        return successes

In [9]:
class BiddingEnvironment:
    def __init__(self, bids, sigma, user_class, price, n_arms):
        self.bids = bids
        self.means = self.initialize_means(user_class=user_class, bids=bids, price=price)
        self.sigmas = np.ones(len(bids)) * sigma
        self.price = price
        self.n_arms = n_arms

    def initialize_means(self, user_class, bids, price):
        means = np.zeros(len(bids))
        for i in range(len(means)):
                means[i] = fun(user_class, bids[i], price)
        return means

    def round(self, pulled_arm):
        return np.random.normal(self.means[pulled_arm], self.sigmas[pulled_arm])

In [13]:

class Learner:
    #Initializes the Learner class with the number of arms, and creates empty lists for rewards per arm and collected rewards.
    def __init__(self, n_arms):
        self.n_arms = n_arms
        self.t = 0
        #list of emty lists, one list for every arm
        self.rewards_per_arm = x = [[] for i in range(n_arms)]
        self.collected_rewards = np.array([])

    # input is pulled arm and environmental reward
    # update the list reward per arm and collect reward
    def update_observations(self, pulled_arm, reward):
        #append the reward value to the list related to the arm
        #pulled by the learn and passed as input
        self.rewards_per_arm[pulled_arm].append(reward)
        self.collected_rewards = np.append(self.collected_rewards, reward)






In [25]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

class GPTS_Learner3(Learner):
    def __init__(self, n_arms, arms):
        super().__init__(n_arms)
        self.arms = arms
        self.means = np.zeros(self.n_arms)
        self.sigmas = np.ones(self.n_arms)*10
        self.pulled_arms = []
        alpha = 10.0
        #kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-3, 1e3))
        #kernel_BlueCow
        kernel = C(100, (100, 1e6)) * RBF(1, (1e-1, 1e1))
        self.gp = GaussianProcessRegressor(kernel=kernel, alpha = alpha**2, normalize_y=True, n_restarts_optimizer= 9)

#Override to update the value of the pulled arms list
    def update_observations(self, arm_idx, reward):
        super().update_observations(arm_idx, reward)
        self.pulled_arms.append(self.arms[arm_idx])
#update the means and sigmas with the new predictions
    def update_model(self):
        x = np.atleast_2d(self.pulled_arms).T
        y = self.collected_rewards
        self.gp.fit(x,y)
        self.means, self.sigmas = self.gp.predict(np.atleast_2d(self.arms).T, return_std=True)
        self.sigmas = np.maximum(self.sigmas, 1e-2)
#calls both update lists
    def update(self, pulled_arm, reward):
        self.t += 1
        self.update_observations(pulled_arm, reward)
        self.update_model()
#returns the index of the max value drawn from the arm normal distribution
    def pull_arm(self, pricing_learner, price_idx, margin):
        if self.t < self.n_arms:
            return self.t  # % self.n_arms

        try:
            conv_rate = pricing_learner.beta_parameters[price_idx, 0] / (pricing_learner.beta_parameters[price_idx, 0]
                                                                         + pricing_learner.beta_parameters[
                                                                             price_idx, 1])
        except ZeroDivisionError:
            conv_rate = 0
            print('DIV 0')
        poisson = pricing_learner.poisson_vector[price_idx, 0] + 1

        exp_rew = np.random.normal(self.acc_means * (np.ones(shape=self.n_arms)
                                                     * margin * conv_rate * poisson - self.cost_means), 50)
        bid_idx = np.argmax(exp_rew)

        return bid_idx
    

In [31]:
for e in tqdm(range(n_experiments), desc='Number of experiments'):
    # Create Learner
    pricing_learner = TS_Learner(n_arms)
    bidding_learner = GPTS_Learner3(n_arms, arms=bids)
    ts_rewards = []

    # generate empty deque
    arms = []
    bids_p = []
    first = True

    # Simulate experiments
    for d in range(T):

        # choose arms
        pricing_pulled_arm = pricing_learner.pull_arm()
        print(pricing_pulled_arm)
        arms.append(pricing_pulled_arm)
        bidding_pulled_arm = bidding_learner.pull_arm(pricing_learner=pricing_learner, price_idx=pricing_pulled_arm,
                                                      margin=calculate_margin(prices[pricing_pulled_arm]))

        # simulate accesses
        # empty daily reward
        ts_daily = {'reward': 0, 'successes': 0, 'clicks': 0}
        costpc = 0

        n_trials, costpc = bid_env.round(pulled_arm=bidding_pulled_arm, user_c=i)
        # quanti acquisti
        ts_successes = pr_env.round(pricing_pulled_arm, c, n_trials)
        # aggiorno beta
        pricing_learner.update(pricing_pulled_arm, ts_successes, n_trials)

        # aggiorno reward_giornaliera
        ts_daily['reward'] += (ts_successes + ts_class_returns) * margin(prices[pricing_pulled_arm]) - n_trials * costpc
        ts_daily['successes'] += ts_successes
        ts_daily['clicks'] += n_trials

        # work on dequy
        ts_dicty = {'arm': pricing_pulled_arm, 'sample': ts_daily['successes']}
        ts_dequy.append(ts_dicty)
        pricing_learner.update_poisson(ts_dequy.popleft())
        bidding_learner.update(pulled_arm=bidding_pulled_arm, costs=costpc, clicks=ts_daily['clicks'])
        # save daily reward
        ts_rewards.append(ts_daily['reward'])
        if check_convergence(arms)[0] and first:
            Didi = {'arm': check_convergence(arms)[1], 'day': d, 'exp': e}
            first = False
            conv_arms.append(Didi)
            # print(check_convergence(arms)[1])
        bids_p.append(bidding_pulled_arm)
        if check_convergence(bids_p, crit=0.7)[0]:
            print('Conv bid', check_convergence(bids_p, crit=0.7)[1])

    final_bids.append(bids_p)
    # print(arms)
    ts_rewards.insert(0, 0)
    ts_final_rewards.append(ts_rewards)
final_bids = np.array(final_bids)



Number of experiments:   0%|          | 0/5 [00:00<?, ?it/s]

53


IndexError: list index out of range

# Trash

In [ ]:
class Pricing_Environment():
    def __init__(self, n_arms, probabilities, prices=None):
        self.n_arms = n_arms
        self.probabilities = probabilities
        self.prices = prices

    def round(self, pulled_arm, user_c, n_trials=1):
        successes = np.random.binomial(n_trials, self.probabilities[user_c.index][pulled_arm])
        number_returns = np.random.poisson(successes * self.poissons[user_c.index])
        marg = self.margins[pulled_arm] * (number_returns + successes)
        return successes, number_returns

In [ ]:
class Bidding_Enviroment():
    def __init__(self, bids, acc_sigma, cost_sigma, user_classes, n_arms):
        self.bids = bids
        self.acc_means = self.initialize_accesses(user_classes=user_classes, bids=bids)
        self.cost_means = self.initialize_cost(user_classes=user_classes, bids=bids)
        self.acc_sigmas = np.ones(len(bids)) * acc_sigma
        self.cost_sigmas = np.ones(len(bids)) * cost_sigma
        self.n_arms = n_arms

    def initialize_accesses(self, user_classes, bids):
        means = np.zeros(shape=(len(user_classes), len(bids)))
        for ii, c in enumerate(user_classes):
            for j, b in enumerate(bids):
                means[ii, j] = c.clicks(b)
        return means

    def initialize_cost(self, user_classes, bids):
        means = np.zeros(shape=(len(user_classes), len(bids)))
        for ii, c in enumerate(user_classes):
            for j, b in enumerate(bids):
                means[ii, j] = cost_per_click(b)
        return means

    def round(self, pulled_arm, user_c):
        sample_accesses = np.random.normal(self.acc_means[user_c, pulled_arm], self.acc_sigmas[pulled_arm])
        sample_cost = np.random.normal(self.cost_means[user_c, pulled_arm], self.cost_sigmas[pulled_arm])
        if sample_cost > self.bids[pulled_arm]:
            sample_cost = self.bids[pulled_arm]
        if sample_cost < 0:
            sample_cost = 0
        if int(sample_accesses) < 0:
            sample_accesses = self.acc_means[user_c, pulled_arm]

        return int(sample_accesses), sample_cost

In [ ]:
def fun(user_class, bid, price):
    mean_per_bid = user_class.get_click_bids(bid) * (user_class.get_conversion_per_price(price) * calculate_margin(price)) - user_class.get_click_bids(bid) * user_class.get_cost_per_click(bid)    #sample_cost
    return mean_per_bid

class BiddingEnvironment:
    def __init__(self, bids, sigma, user_class, price, n_arms):
        self.bids = bids
        self.means = self.initialize_means(user_class=user_class, bids=bids, price=price)
        self.sigmas = np.ones(len(bids)) * sigma
        self.price = price
        self.n_arms = n_arms

    def initialize_means(self, user_class, bids, price):
        means = np.zeros(len(bids))
        for i in range(len(means)):
                means[i] = fun(user_class, bids[i], price)
        return means

    def round(self, pulled_arm):
        return np.random.normal(self.means[pulled_arm], self.sigmas[pulled_arm])